In [ ]:
number_text = "55119xxxxxxxx"
name_text = "Anotações"
image_path = r"C:\Users\brrochana1\Repositories\cleoenvia\src\gooey-images\config_icon.png"
message = "Teste 👌"

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import os
import pickle
from io import BytesIO
import win32clipboard
from PIL import Image


def send_to_clipboard(data, clip_type="text"):
    win32clipboard.OpenClipboard()
    try:
        win32clipboard.EmptyClipboard()

        if clip_type == "text":
            clip_type = win32clipboard.CF_UNICODETEXT
        elif clip_type == "image":
            clip_type = win32clipboard.CF_DIB
        win32clipboard.SetClipboardData(clip_type, data)
    finally:
        win32clipboard.CloseClipboard()


os.environ["WDM_SSL_VERIFY"] = "false"

notebook_folder = os.path.dirname(os.path.abspath("__file__"))

selenium_folder = os.path.join(notebook_folder, "selenium")

os.makedirs(selenium_folder, exist_ok=True)

cookies_file = os.path.join(selenium_folder, "cookies.pkl")

In [ ]:
_pillow_image = Image.open(image_path)

_bytes_output_image = BytesIO()
_pillow_image.convert("RGB").save(_bytes_output_image, "BMP")

image_data = _bytes_output_image.getvalue()[14:]
_bytes_output_image.close()

In [ ]:
options = webdriver.EdgeOptions()
options.add_argument(f"user-data-dir={selenium_folder}")

driver = webdriver.Edge(
    service=EdgeService(EdgeChromiumDriverManager().install()), options=options
)

if os.path.exists(cookies_file):
    with open(cookies_file, "rb") as f:
        cookies = pickle.load(f)
        for cookie in cookies:
            driver.add_cookie(cookie)

actions = ActionChains(driver)

In [ ]:
def check(xpath, operator=None, *other_conditions):
    return (
        WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        if operator is None
        else WebDriverWait(driver, 30).until(
            operator(
                EC.element_to_be_clickable((By.XPATH, xpath)),
                *[
                    condition((By.XPATH, xpath), *args, **kwargs)
                    for condition, args, kwargs in other_conditions
                ],
            )
        )[0]
    )


def find_and_click(xpath, operator=None, *other_conditions):
    element = check(xpath, operator, *other_conditions)
    element.click()
    return element

In [ ]:
driver.get(
    "https://web.whatsapp.com"
)  # /send?phone=55119xxxxxxxx&text&source&data&app_absent')

In [ ]:
xpath_search_enter = r'//*[@id="side"]/div[1]/div/div/div[2]'

find_and_click(xpath_search_enter)

In [ ]:
xpath_search_input = r'//*[@id="side"]/div[1]/div/div/div[2]/div/div[1]/p'

element_search_input = find_and_click(xpath_search_input)

In [ ]:
if input_length := len(element_search_input.text):
    actions.send_keys(Keys.BACKSPACE * input_length)
    actions.perform()
    element_search_input = find_and_click(xpath_search_input)

In [ ]:
xpath_search_input = r'//*[@id="side"]/div[1]/div/div/div[2]/div/div[1]/p'

element_search_input = find_and_click(xpath_search_input)

In [ ]:
actions.send_keys_to_element(element_search_input, number_text)
actions.perform()

In [ ]:
assert element_search_input.text == number_text

In [ ]:
xpath_contact_result_title = (
    f'//*[@id="pane-side"]/div[1]/div/div//span[@title="{name_text}"]'
)
find_and_click(xpath_contact_result_title)

In [ ]:
xpath_contact_header = r'//*[@id="main"]/header/div[2]/div[1]/div/span[1]'

check(
    xpath_contact_header,
    EC.all_of,
    (EC.text_to_be_present_in_element, (name_text,), {}),
)

In [ ]:
xpath_input_message_only = (
    r'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[1]/p'
)

element_input_message_only = check(xpath_input_message_only)

In [ ]:
send_to_clipboard(image_data, win32clipboard.CF_DIB)

In [ ]:
element_input_message_only.send_keys(Keys.CONTROL, "v")

In [ ]:
xpath_one_image_preview_container = r'//*[@id="app"]/div/div/div[3]/div[2]/span/div/span/div/div/div[2]/div/div[2]/div[1]/div[count(child::*) = 1]'

check(xpath_one_image_preview_container)

In [ ]:
xpath_input_image_description = r'//*[@id="app"]/div/div/div[3]/div[2]/span/div/span/div/div/div[2]/div/div[1]/div[3]/div/div/div[2]/div[1]/div[1]/p'

element_input_image_description = check(xpath_input_image_description)

In [ ]:
if input_length := len(element_input_image_description.text):
    actions.send_keys(Keys.BACKSPACE * input_length)
    actions.perform()
    element_input_image_description = check(xpath_input_image_description)

In [ ]:
send_to_clipboard(message)

In [ ]:
element_input_image_description.send_keys(Keys.CONTROL, "v")

In [ ]:
assert element_input_image_description.text == message

In [ ]:
xpath_send_button = r'//*[@id="app"]/div/div/div[3]/div[2]/span/div/span/div/div/div[2]/div/div[2]/div[2]/div/div'

find_and_click(xpath_send_button)

In [ ]:
WebDriverWait(driver, 30).until_not(EC.element_to_be_clickable((By.XPATH, xpath_send_button)))

In [ ]:
with open(cookies_file, "wb") as f:
    pickle.dump(driver.get_cookies(), f)